In [1]:
# !sudo python3 -m pip install tensorflow==2.0.0-alpha0

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn import model_selection, metrics
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras
import glob


import os
print(os.listdir("."))

# Any results you write to the current directory are saved as output.

['tf2_model2-Boosted.ipynb', '.ipynb_checkpoints', 'test.csv', 'train.csv', 'submission3.csv', 'Keras_-_PUBG.ipynb', 'sample_submission.csv', 'tf2_model1.ipynb', 'estimator_submission.csv', 'estimator_api.ipynb', 'PUBG_submit.ipynb', 'Esti_works.ipynb', 'tf2_model3-Keras.ipynb', 'EDA.ipynb', 'Untitled.ipynb']


Using TensorFlow backend.


In [3]:
def load_data(train_path='train.csv', test_path='test.csv'):
    train = pd.DataFrame.from_csv(train_path, index_col="Id")
    test = pd.DataFrame.from_csv(test_path, index_col="Id")
#     train = train.reindex(np.random.permutation(train.index))
    return train, test

In [4]:
train, test = load_data()

/home/lazukav/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/home/lazukav/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
def preprocess(data):
    data.dropna(axis=0, inplace=True)
    if 'winPlacePerc' in data:
        data.winPlacePerc = np.where(data.winPlacePerc >= 0.5 , 1, 0)
    return data

In [6]:
train = preprocess(train)
test = preprocess(test)

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train.drop('winPlacePerc', axis=1), train.winPlacePerc, test_size=0.2, shuffle=None)

In [8]:
# X_train.select_dtypes(exclude=['object']).columns.values

In [9]:
# cols = [tf.feature_column.numeric_column(i) for i in X_train.select_dtypes(exclude=['object']).columns.values]
# for i in train.select_dtypes(include=['object']).columns.values:
#     cols.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key=i, vocabulary_list=train[i].unique())))
# cols.append(matchType)
cols = [tf.feature_column.numeric_column(i) for i in ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills']]

In [10]:
# def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
#     def input_function():
#         ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
#         if shuffle:
#             ds = ds.shuffle(1000)
#         ds = ds.batch(batch_size).repeat(num_epochs)
#         return ds
#     return input_function

In [11]:
def train_classification_model(
    learning_rate,
    steps,
    batch_size,
    X_train,
    y_train,
    X_test,
    y_test,
    cols,
    m_dir,
    periods):

    steps_per_period = steps / periods  

#     training_input_fn = make_input_fn(X_train, y_train, shuffle=True)

#     ptraining_input_fn = make_input_fn(X_train, y_train, num_epochs=1, shuffle=False)

#     validation_input_fn = make_input_fn(X_test, y_test, num_epochs=1, shuffle=False)

    training_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = X_train,
        y = y_train,
        shuffle = True,
    )
    
    ptraining_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = X_train,
        y = y_train,
        num_epochs = 1,
        shuffle = False,
    )
    
    validation_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = X_test,
        y = y_test,
        num_epochs = 1,
        shuffle = False,
    )

    classifier = tf.estimator.DNNClassifier(
        feature_columns=cols,
        n_classes=2,
        hidden_units=[64, 128, 64],
        model_dir=m_dir
    )

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
#     print("LogLoss error (on validation data):")
    training_errors = []
    validation_errors = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        classifier.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        classifier.evaluate(input_fn=validation_input_fn)
        print("period:", period + 1)
    print("Done")
    return classifier

In [16]:
# %%time

model = train_classification_model(
    learning_rate=0.005,
    steps=100,
    batch_size=10,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    cols=cols,
    periods=5,
    m_dir='/tmp/model1')

Training model...
LogLoss error (on validation data):
period: 1
period: 2


In [19]:
# %%time

scores1 = model.predict(input_fn=tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = test,
    num_epochs = 1,
    shuffle = False,
))
print("done!")
res = []
for i, p in enumerate(scores1):
#     print(i)
    res.append(p['class_ids'][0])   

done!


In [20]:
submission = pd.DataFrame(
    {"Id": test.index, "winPlacePerc": res},
    columns=["Id", "winPlacePerc"]
)
submission.to_csv("submission1.csv", index=False)